In [1]:
%%writefile functions.py

import numpy as np
import scipy as sp
import scipy.special as sps
import matplotlib.pyplot as plt

def mode_function(kappa, gamma, t0, t):
    return kappa*np.exp(-gamma*abs(t-t0)) - gamma*np.exp(-kappa*abs(t-t0))

def factorial(n):
    fact = 1
    if n == 0:
        return fact
    else:
        for i in range(n):
            fact *= i+1
    return fact

def a_operator(n_dim):
    a_operator = np.zeros((n_dim+1, n_dim+1))
    for j in range(n_dim):
        a_operator[j, j+1] = np.sqrt(j+1)
    return a_operator
        
def dagger(M):
    if type(M) == np.ndarray:
        dag = np.asarray(np.matrix(M).H)
    elif type(M) == np.matrix:
        dag = M.H
    else:
        print('Object is not a matrix or array')
    return dag

def wigner_function(density_matrix, n_dim):
    '''
    Compute Wigner function for specified dimension based on the density matrix given.
    '''
    q, p = [np.linspace(-10, 10, 200) for j in range(2)]
    P, Q = np.meshgrid(np.atleast_1d(p), np.atleast_1d(q))
    W = np.zeros((len(p), len(q)), dtype=complex)
    W_nm = np.zeros((len(p), len(q), n_dim+1, n_dim+1), dtype=complex) #expansion coefficients of the Wigner function
    X = 2*(Q**2 + P**2)
    #Compute the lower triangle of the matrix W_nm
    for n in np.arange(n_dim+1):
        for m in np.arange(n+1):
            dif = float(n-m)
            W_nm[:, :, n, m] = (1/np.pi)*np.exp(-0.5*X)*(-1)**m \
                               *(Q-1j*P)**dif*np.sqrt(2**dif*sp.special.gamma(m+1)/sp.special.gamma(n+1))\
                               *sp.special.assoc_laguerre(x=X, n=m, k=dif)     
    #Compute the upper triangle without the diagonal
    for m in np.arange(n_dim+1):
        for n in np.arange(m):
            W_nm[:, :, n, m] = np.conj(W_nm[:, :, m, n])

    #Compute the Wigner function
    for n in np.arange(n_dim+1):
        for m in np.arange(n_dim+1):
            W += W_nm[:, :, n, m]*density_matrix[n, m]
    #Get rid of nan values
    shape = W.shape
    W = W.flatten()
    W[np.where(np.isnan(W))] = 0
    W = W.reshape(shape)
    #Normalize in L1
    dq = q[1] - q[0]
    dp = p[1] - p[0]
    W *= np.pi/np.sum(np.sum(W) * dq*dp)
    
    return Q, P, W

def psi_coherent(alpha, theta, dimension):
    '''
    Compute 1 mode wavefunction for a coherent state
    Parameters:
        alpha: float
            Amplitude of the state
        theta: float
            Angle of the state in degrees
        dimension: int
            Fock state dimension of the computation
    Return
        psi: 1D array
            Wavefunction
    '''
    return [np.exp(-np.abs(alpha)**2/2)*(alpha*np.exp(1j*theta*np.pi/180))**n/np.sqrt(factorial(n)) for n in range(dimension+1)]

def rho_psi(psi):
    '''
    Compute 1 mode density matrix based on wavefunction
    Parameters:
        psi: 1D array
            Wavefunction
    Return:
        rho: 2D array
            Density matrix
    '''
    dimension = len(psi)
    rho = np.zeros((dimension,dimension), dtype=complex)
    for i in range(dimension):
        for j in range(dimension):
            rho[i, j] = psi[i]*np.conjugate(psi[j])
    return rho/np.trace(rho)

def iyad_homodyneFockPOVM(n_max, x, theta):
    """
    This function computes, the homodyne detection POVM in the Fock basis, up
    to order "N" (i.e, in a reduced-dimension Hilbert space). 
    
    For a quadrature value x, the (n, m) element of the POVM matrix is
    <X|m>*<X|n>. The expression of <X|k> depends on the associated LO phase
    "theta"
    
    n_max: positive integer
    x: scalar
    theta: scalar
    """
    #For each element of x, a POVM matrix is computed
    if n_max < 2:
        raise InvalidDimensionError('The Hilbert space dimension must be at least 2')
    POVM = np.zeros((n_max+1, ), dtype=complex)
    for n in np.arange(n_max):
        if n==0:
            POVM[n] = 1/(np.sqrt(np.sqrt(np.pi)))*np.exp(-0.5*x**2)
        elif n==1:
            POVM[n] = x*np.sqrt(2)*np.exp(-1j*theta) * POVM[0]
        else:
            POVM[n] = np.exp(-1j*theta)/np.sqrt(n)*(np.sqrt(2)*x*POVM[n-1] - np.exp(-1j*theta)*np.sqrt(n-1)*POVM[n-2])
    return np.tensordot(POVM, np.conj(POVM), axes=0)

def jonas_homodyneFockPOVM(n_max, x, theta):
    """
    This function computes the homodyne detection POVM in the Fock basis up
    to order "N" (i.e, in a reduced-dimension Hilbert space) using the Hermite polynomials.
    
    For a quadrature value x, the (n, m) element of the POVM matrix is
    <X|m>*<X|n>. The expression of <X|k> depends on the associated LO phase
    "theta"
    
    n_max: positive integer
    x: scalar
    theta: scalar
    """
    #For each element of x, a POVM matrix is computed
    if n_max < 2:
        raise InvalidDimensionError('The Hilbert space dimension must be at least 2')
    POVM = np.zeros((n_max+1, ), dtype=complex)
    for n in np.arange(n_max):
        POVM[n] = np.exp(-1j*n*theta)*sps.hermite(n)(x)*np.exp(-x*x/2.)/np.sqrt(np.sqrt(np.pi)*2**n*sps.gamma(n+1))
    return np.tensordot(POVM, np.conj(POVM), axes=0)
    

def isDensityMatrix(M, tolerance=1/100):
    """
    This function checks whether the matrix is a density matrix.
    M is a density matrix if and only if:

        - M is Hermitian
        - M is positive-semidefinite
        - M has trace 1
    """
    check_details = {}
    eigenvalues = np.linalg.eigvals(M)
    max_eigenval = np.max(np.abs(eigenvalues))
    check_details['is Hermitian'] = not np.any([not np.isclose(np.imag(e), 0) for e in eigenvalues])
    check_details['is positive-semidefinite'] = np.all([np.real(e)>=-tolerance*max_eigenval for e in eigenvalues])
    check_details['has trace 1'] = np.isclose(np.trace(M), 1)
    check = check_details['is Hermitian'] and check_details['is positive-semidefinite'] \
            and check_details['has trace 1']
    return check, check_details

def traceDistance(M_1, M_2):
    """
    Source: https://en.wikipedia.org/wiki/Trace_distance
    
    INPUTS
    ------------
        M_1 : 2-D array-like of complex
            Matrix 1
        M_2 : 2-D array-like of complex
            Matrix 2
    
    OUTPUTS
    -----------
    The trace distance
    """   
    X = M_1 - M_2
    return 0.5*np.trace(sp.linalg.sqrtm(np.transpose(np.conj(X)) @ X))


def quantumStateFidelity(rho_1, rho_2):
        """
        This function computes the fidelity of two quantum states, defined by
        the density matrices 'rho_1' and 'rho_2'.
        
        Source: https://en.wikipedia.org/wiki/Fidelity_of_quantum_states
        
        INPUTS
        ------------
            rho_1 : 2-D array-like of complex
                Density matrix 1
            rho_2 : 2-D array-like of complex
                Density matrix 2
        
        OUTPUTS
        -----------
        The fidelity (float in [0, 1])
        """                
        X = sp.linalg.sqrtm(rho_1) @ sp.linalg.sqrtm(rho_2)
        return (2*traceDistance(X, 0))**2

def hasConverged(rho_1, rho_2, convergence_parameter):
    convergence_metrics = np.real(quantumStateFidelity(rho_1, rho_2))
    converged = convergence_metrics > 1-convergence_parameter
    return converged, convergence_metrics

def calculate_projection_operators(data, dimension, tomography_phases = ['000', '030', '060', '090', '120', '150'], n_bins = 200):
    # Organize the quadrature measurements into bins
    # Compute homodyne projection operators in Fock basis

    # Maximum and minimum quadrature values for plotting
    quad_max = [np.max(data[phase]) for phase in tomography_phases]
    quad_max = np.max(quad_max)
    quad_min = [np.min(data[phase]) for phase in tomography_phases]
    quad_min = np.min(quad_min)

    x_bin_edges = np.linspace(quad_min, quad_max, n_bins+1)

    # Computing of projection operators and organization of quadrature measurements
    projection_operators = np.zeros((dimension+1, dimension+1, n_bins, len(tomography_phases)), dtype=complex)
    n_observations = np.zeros((n_bins, len(tomography_phases)))

    n_x = 1
    for p, phase in enumerate(tomography_phases):
        for j in range(n_bins):
            x = data[phase].flatten()
            data_bin = x[np.where(np.logical_and(x>x_bin_edges[j], x<x_bin_edges[j+1]))]
            x_bin_continuous = np.linspace(x_bin_edges[j], x_bin_edges[j+1], n_x)
            n_observations[j, p] = len(data_bin)
            projection_operator = np.zeros((dimension+1, dimension+1), dtype=complex)
            for k in range(n_x):
                jonas_projection_operator = jonas_homodyneFockPOVM(n_max = dimension, x = x_bin_continuous[k], theta = p)
                iyad_projection_operator = iyad_homodyneFockPOVM(n_max = dimension, x = x_bin_continuous[k], theta = p)
                projection_operator += iyad_projection_operator
            projection_operators[:,:,j,p] = (x_bin_edges[j+1] - x_bin_edges[j])*projection_operator
    return projection_operators, n_observations
            
def apply_maximum_likelihood(data, dimension, tomography_phases = ['000', '030', '060', '090', '120', '150'], convergence_parameter = 1e-8, n_bins = 200):
    # Apply the maximum likelihood method
    # Code is based on Iaji.Physics.Theory.QuantumMechanics.SimpleHarmonicOscillator.QuantumStateTomography

    converged = False
    projection_operators, n_observations = calculate_projection_operators(data, dimension, n_bins = n_bins)
    
    rho = (1/(dimension+1))*np.eye(dimension+1, dtype=complex) #estimated density operator
    past_log_likelihood = []
    run = 0
    while not converged:
        run += 1
        rho_last = rho
        R = np.zeros((dimension+1, dimension+1), dtype=complex)
        measurement_probabilities = np.zeros((n_bins, len(tomography_phases)))
        log_likelihood = 0
        for p in range(len(tomography_phases)):
            for j in range(n_bins):
                measurement_probabilities[j, p] = np.real(np.trace(projection_operators[:, :, j, p] @ rho))
        measurement_probabilities /= len(tomography_phases)
        R = np.tensordot(n_observations/measurement_probabilities, projection_operators, axes=([0, 1], [2, 3]))              
        R /= np.trace(R)
        log_likelihood = np.sum(n_observations*np.log(measurement_probabilities), (0, 1))
        R = (R+np.transpose(np.conj(R)))/2
        rho = R @ (rho @ R)
        rho /= np.trace(rho)
        #Check that it is a density matrix
        is_density_matrix = isDensityMatrix(rho)
        if not is_density_matrix[0]:
            raise NotADensityMatrixError(str(is_density_matrix[1]))
        past_log_likelihood.append(log_likelihood)
        #Check convergence
        converged, convergence_metrics = hasConverged(rho_1 = rho, rho_2 = rho_last, convergence_parameter=convergence_parameter)
    print('Converged')
    print('%d runs'%run)
    
    return rho

def calculate_output_state(data, input_rho, success_rate, dimension, plot = True, verbose = True):
    rho = apply_maximum_likelihood(data, dimension)
    
    # Calculate |alpha| and theta
    annihilation = a_operator(dimension)
    displacement = np.trace(annihilation @ rho)
    alpha = np.abs(displacement)
    angle = np.arctan(displacement.imag/displacement.real)*180/np.pi
    print('|alpha| = %.2f'%alpha)
    print('theta = %.2f°'%angle)
    
    if plot:
        # Plot density matrix
        plt.figure(figsize=(11, 8))
        #Define the maximum modulus of the density operator
        rho_max = np.max(np.abs(rho))
        #Plot
        plt.imshow(np.abs(rho), norm=matplotlib.colors.Normalize(vmin=-rho_max, vmax=rho_max))    
        plt.xlabel("n")
        plt.ylabel("m")
        plt.title("Density matrix")
        plt.pause(.05)
        # plt.colorbar()
        plt.show()
        print('Correlation term is', rho[0,1])

        # Plot photon number distribution
        figure = plt.figure(figsize=(11, 8))
        number = np.linspace(0, dimension, dimension+1)
        photon_number_distribution = [np.abs(rho[j, j]) for j in range(rho.shape[0])]
        #Plot
        plt.bar(number, photon_number_distribution)
        plt.xlabel("Photon number")
        plt.ylabel("Probability")
        plt.title("Output photon number distribution")
        plt.pause(.05)
        plt.show()
        print('Amount of single-photons:',photon_number_distribution[1])

        # Calculate Wigner function
        Q, P, W = wigner_function(rho, dimension)

        # Plot Wigner function
        W_max = np.max(np.abs(W))

        fig = plt.figure(figsize = [13,9])
        ax = plt.axes(projection='3d')
        W = np.real(W).astype(float)
        Q = Q.astype(float)
        P = P.astype(float)
        ax.plot_surface(Q, P, W, cmap='viridis', norm=matplotlib.colors.Normalize(vmin=-W_max, vmax=W_max))
        ax.set_title('Output state Wigner function')
        ax.set_xlabel('q')
        ax.set_ylabel('p')
        ax.set_zlabel('W(q,p)')
        fig.add_axes(ax)
        plt.show()

        # Countour plot of Wigner function
        fig = plt.figure(figsize=(9, 9) )
        ax = plt.axes()
        ax.contourf(Q, P, W, cmap='viridis', norm=matplotlib.colors.Normalize(vmin=-W_max, vmax=W_max))
        ax.set_title('Output state Wigner function')
        ax.set_xlabel('q')
        ax.set_ylabel('p')
        ax.grid()
        fig.add_axes(ax)
        #fig.colorbar(ax)
        plt.show()

    fidelity = quantumStateFidelity(rho, input_rho)
    if verbose:
        print("Output fidelity is %f" %fidelity)
        print("Success rate is %f" %np.mean(success_rate))
        print("The product of these values is %f" %(fidelity*np.mean(success_rate)))
        print("Purity of output state is %.2f" %np.trace(rho@rho))
    
    return rho, fidelity

Overwriting functions.py


In [2]:
%%writefile rawdata.py

import numpy as np
from scipy.fft import fft, fftfreq
import os
from qopt import lecroy
import matplotlib.pyplot as plt
import functions
import inputstate

class rawdata:
    def __init__(self, path, dimension, tomography_phases = ['000', '030', '060', '090', '120', '150'], phase_vacuum = True):
        self.path = path
        self.dimension = dimension
        self.tomography_phases = tomography_phases
        self.phase_vacuum = phase_vacuum
        self.homodyne, self.heralding, self.charlie, self.vacuum, self.meta = self.import_data()

    def import_data(self):
        '''
        Import teleportation data
        Parameters
            path: string
                folder on which data is saved
            phase_vacuum: boolean
                True if each vacuum measurement is associated with a different phase angle
        Returns

        '''
        homodyne_files = {self.tomography_phases[i]: \
                          np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C1tele" + self.tomography_phases[i] in f]) \
                          for i in range(len(self.tomography_phases))}
        heralding_files = {self.tomography_phases[i]: \
                          np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C2tele" + self.tomography_phases[i] in f]) \
                          for i in range(len(self.tomography_phases))}
        charlie_files = {self.tomography_phases[i]: \
                          np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C3tele" + self.tomography_phases[i] in f]) \
                          for i in range(len(self.tomography_phases))}
        if self.phase_vacuum:
            vacuum_files = {self.tomography_phases[i]: \
                              np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C1vac" + self.tomography_phases[i] in f]) \
                              for i in range(len(self.tomography_phases))}
        else:
            vacuum_files = {self.tomography_phases[i]: \
                              np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C1vac" in f]) \
                              for i in range(len(self.tomography_phases))}

        self.files_for_each_phase = len(homodyne_files[self.tomography_phases[0]])

        homodyne = {phase: lecroy.read(homodyne_files[phase][0])[2] for phase in self.tomography_phases}
        heralding = {phase: lecroy.read(heralding_files[phase][0])[2] for phase in self.tomography_phases}
        charlie = {phase: lecroy.read(charlie_files[phase][0])[2] for phase in self.tomography_phases}
        vacuum = {phase: lecroy.read(vacuum_files[phase][0])[2] for phase in self.tomography_phases}
        if self.files_for_each_phase > 1:
            for _, phase in enumerate(self.tomography_phases):
                for j in range(self.files_for_each_phase-1):
                    homodyne[phase] = np.concatenate((homodyne[phase], lecroy.read(homodyne_files[phase][j+1])[2]))
                    heralding[phase] = np.concatenate((heralding[phase], lecroy.read(heralding_files[phase][j+1])[2]))
                    charlie[phase] = np.concatenate((charlie[phase], lecroy.read(charlie_files[phase][j+1])[2]))
                if len(vacuum_files[phase]) > 1:
                    vacuum[phase] = np.concatenate((vacuum[phase], lecroy.read(vacuum_files[phase][0])[2]))
        
        # Extract information from metadata
        meta = lecroy.read(heralding_files[self.tomography_phases[0]][0])[0]
        self.sequences = meta['subarray_count']
        self.total_sequences = self.files_for_each_phase*self.sequences
        self.total_points = meta['wave_array_count']
        self.points_per_seq = int(meta['wave_array_count']/self.sequences)
        self.Ts = meta['horiz_interval']
        
        # Electronic noise
        elec_file = np.array([os.path.join(self.path, f) for f in sorted(os.listdir(self.path)) if "C1elec" in f])
        elec = lecroy.read(elec_file[0])[2]

        print('Data imported')
        self.check_clearance(homodyne, vacuum, elec)

        return homodyne, heralding, charlie, vacuum, meta

    def check_clearance(self, homodyne, vacuum, elec):
        tr000 = homodyne[self.tomography_phases[0]]
        trvac = vacuum[self.tomography_phases[0]]
        trelec = elec
        freq = fftfreq(502, .002)

        plt.figure(figsize=(10,6))
        plt.plot(freq[:60], 10*np.log10((np.abs(fft(trvac))**2).mean(0))[:60], color='orange', label='Vacuum')
        plt.plot(freq[:60], 10*np.log10((np.abs(fft(tr000))**2).mean(0))[:60], color='green', label='Signal')
        plt.plot(freq[:60], 10*np.log10((np.abs(fft(trelec))**2).mean(0))[:60], color='blue', label='Electronic')
        plt.legend()
        plt.grid()
        plt.xlabel('MHz')
        plt.ylabel('Noise power [dB]')

        # Calculate clearance for a desired frequency
        freq_clear = 10
        min_f = [np.abs(f - 10) for f in freq[:60]]
        min_p = np.where(min_f == np.min(min_f))[0][0]

        vac_clear = 10*np.log10((np.abs(fft(trvac))**2).mean(0))[:60][min_p]
        elec_clear = 10*np.log10((np.abs(fft(trelec))**2).mean(0))[:60][min_p]

        clearance = vac_clear - elec_clear
        print('Clearance at %.1f MHz is %.2f dB' %(freq_clear, clearance))

    def fluctuations_conference(self, conf_homodyne, conf_vacuum, mf):
        '''
        Check fluctuations suffered by data during measurements
        Parameters
            homodyne: dict
                raw measurement data
            vacuum: dict
                raw vacuum data
        '''
        # Conference performed for raw data with mode function applied
        conf_vacuum = {phase: np.zeros((self.sequences)) for phase in self.tomography_phases}
        conf_homodyne = {phase: np.zeros((self.total_sequences)) for phase in self.tomography_phases}

        for i, phase in enumerate(self.tomography_phases):
            for k in range(self.total_sequences):
                conf_homodyne[phase][k] = np.dot(homodyne[phase][k], self.mf)
                if k < sequences:
                    conf_vacuum[phase][k] = np.dot(vacuum[phase][k], self.mf)

        print('CONFERENCE OF FLUCTUATIONS OF OUTPUT DATA')
        #Homodyne data
        w = 200
        fig, axs = plt.subplots(3,2, figsize=(15,12), sharey=True, sharex=True)
        for i, ax in enumerate(axs.flat):
            ax.plot(conf_homodyne[self.tomography_phases[i]], '.', alpha=.1)
            ax.plot(np.convolve(conf_homodyne[self.tomography_phases[i]], np.ones(w), 'same') / w)
            div = make_axes_locatable(ax)
            axB = div.append_axes("right", 1, pad=0.1, sharey=ax)
            axB.xaxis.set_tick_params(labelbottom=False)
            axB.yaxis.set_tick_params(labelleft=False)
            axB.text(.02, 3.5, 'var.: {:.2f}'.format(conf_homodyne[self.tomography_phases[i]].var()))
            bins = np.linspace(-4,4,101)
            axB.hist(conf_homodyne[self.tomography_phases[i]], bins=bins, density=True, orientation='horizontal')
            plt.title('Homodyne measurement phase '+ self.tomography_phases[i], loc = 'center')

        #Vacuum data
        fig, axs = plt.subplots(3,2, figsize=(15,12), sharey=True, sharex=True)
        for i, ax in enumerate(axs.flat):
            ax.plot(conf_vacuum[self.tomography_phases[i]], '.', alpha=.1)
            ax.plot(np.convolve(conf_vacuum[self.tomography_phases[i]], np.ones(w), 'same') / w)
            div = make_axes_locatable(ax)
            axB = div.append_axes("right", 1, pad=0.1, sharey=ax)
            axB.xaxis.set_tick_params(labelbottom=False)
            axB.yaxis.set_tick_params(labelleft=False)
            axB.text(.02, 3.5, 'var.: {:.2f}'.format(conf_vacuum[self.tomography_phases[i]].var()))
            bins = np.linspace(-4,4,101)
            axB.hist(conf_vacuum[self.tomography_phases[i]], bins=bins, density=True, orientation='horizontal')
            plt.title('Vacuum measurement phase '+ self.tomography_phases[i], loc = 'center')

    def offset_correct(self, file, block_size = 100):
        '''
        Correct offset by considering last few points as vacuum.
        Parameters:
            file: dictionary
                data organized by phase
            block_size: int
                amount of points for which data will be divided
        Return:
            file_corr: corrected data
        '''
        seq = len(file['000'])
        file_corr = {}
        block_size = 100
        for k,v in file.items():
            tails = v[:,-(self.points_per_seq//5):].reshape((seq//block_size, block_size, -1))
            tail_avg = tails.mean((1,2))
            _file_corr = v.reshape((seq//block_size, block_size, -1)) - tail_avg[:, np.newaxis, np.newaxis]
            _file_corr = _file_corr.reshape((seq, self.points_per_seq))
            file_corr[k] = _file_corr
        return file_corr

    def plot_mode_function(self, file, homodyne_g = 2*np.pi*17e6, homodyne_k = 2*np.pi*30e6, homodyne_time_delay = -250e-9, correct = False):
        '''
        Plot mode function along with variance across each data point to check fitting.
        Parameters:
            file: dictionary
                data organized by phase
        Return:
            mf: 1D array
                mode function
        '''
        # Correct data offset
        if correct:
            file_corr = self.offset_correct(file)
        else:
            file_corr = file

        # Mode function with time information from data
        begin = self.meta['horiz_offset']
        x = np.linspace(begin, begin + self.Ts*self.total_points, self.total_points)
        x_sequence = np.linspace(begin, begin + self.Ts*self.points_per_seq, self.points_per_seq)
        self.mf = functions.mode_function(homodyne_k, homodyne_g, homodyne_time_delay, x_sequence)

        seq_var = np.array([np.var(file_corr[phase], axis=0) for phase in self.tomography_phases])
        mean_seq_var = np.mean(seq_var)
        mf_norm = (self.mf - self.mf.mean())*(seq_var.mean(axis=0).max().max() - mean_seq_var)/(self.mf.max() - self.mf.mean() + mean_seq_var)

        # Plot variance of data and normalized mode function
        print('VARIANCE ACROSS EACH POINT OF TELEPORTED DATA')
        plt.figure()
        plt.plot(x_sequence, seq_var.mean(axis=0) - mean_seq_var)
        plt.legend()
        plt.plot(x_sequence, mf_norm)
        plt.title('Variance of measured signal')
        plt.xlabel('Time (s)')
        plt.ylabel('Variance (V²)')
        plt.show()

        return self.mf

    def apply_mode_function(self, homodyne, vacuum, homodyne_g = 2*np.pi*17e6, homodyne_k = 2*np.pi*30e6, homodyne_time_delay = -250e-9, input_folder = None, correct = True, check_fluctuations = False, input_plot = True):
        '''
        Apply temporal mode function to the data.
        Data is corrected and normalized to vacuum shot noise (1/2).
        Parameters
            input_folder: str
                if not None, the input state is also analysed
            homodyne: dictionary
                homodyne measurements
            vacuum: dictionary
                vacuum measurements
        Return:
            mf_homodyne: dictionary
                homodyne measurements with mode function applied
            mf_vacuum: dictionary
                vacuum measurements with mode function applied
        '''
        if correct:
            H_corr = self.offset_correct(homodyne)
            V_corr = self.offset_correct(vacuum)
        else:
            H_corr = homodyne
            V_corr = vacuum

        self.mf = self.plot_mode_function(homodyne)

        if check_fluctuations:
            self.fluctuations_conference(homodyne, vacuum)

        # Apply mode function to data
        mf_vacuum = {phase: np.zeros((self.sequences)) for phase in self.tomography_phases}
        mf_homodyne = {phase: np.zeros((self.total_sequences)) for phase in self.tomography_phases}
        seq_var = {phase: np.zeros((self.total_sequences)) for phase in self.tomography_phases}

        for i, phase in enumerate(self.tomography_phases):
            for k in range(self.total_sequences):
                mf_homodyne[phase][k] = np.dot(H_corr[phase][k], self.mf)
                if k < self.sequences:
                    mf_vacuum[phase][k] = np.dot(V_corr[phase][k], self.mf)
            seq_var[phase] = np.var(H_corr[phase], axis = 0)

        # Normalize data to vacuum level
        mean_vac = {phase: np.mean(mf_vacuum[phase]) for phase in self.tomography_phases}
        var_vac = {phase: np.var(mf_vacuum[phase]) for phase in self.tomography_phases}

        for i, phase in enumerate(self.tomography_phases):
            mf_homodyne[phase] = (mf_homodyne[phase] - mean_vac[phase])/np.sqrt(2*var_vac[phase])
            mf_vacuum[phase] = (mf_vacuum[phase] - mean_vac[phase])/np.sqrt(2*var_vac[phase])

        if input_folder == None:
            return mf_homodyne, mf_vacuum
        else:
            input_rho = inputstate.reconstruct_input_state(input_folder, self.mf, input_plot = input_plot)
            return mf_homodyne, mf_vacuum, input_rho

Overwriting rawdata.py


In [10]:
%%writefile inputstate.py

import numpy as np
from scipy.fft import fft, fftfreq
import os
import matplotlib
import matplotlib.pyplot as plt
from qopt import lecroy
import functions

class inputstate:
    
    def __init__(self, input_folder, dimension, modefunction, input_plot = True):
        self.input_folder = input_folder
        self.dimension = dimension
        self.mf = modefunction
        self.input_plot = input_plot
        self.input_rho = self.reconstruct_input_state()
        
    def check_clearance(self, input_data, elec, plot = True):
        tr000 = input_data['000']
        trelec = elec
        freq = fftfreq(502, .002)

        plt.figure(figsize=(10,6))
        plt.plot(freq[:60], 10*np.log10((np.abs(fft(tr000))**2).mean(0))[:60], color='green', label='Signal')
        plt.plot(freq[:60], 10*np.log10((np.abs(fft(trelec))**2).mean(0))[:60], color='blue', label='Electronic')
        plt.legend()
        plt.grid()
        plt.xlabel('MHz')
        plt.ylabel('Noise power [dB]')

        # Calculate clearance for a desired frequency
        freq_clear = 10
        min_f = [np.abs(f - 10) for f in freq[:60]]
        min_p = np.where(min_f == np.min(min_f))[0][0]

        tr_clear = 10*np.log10((np.abs(fft(tr000))**2).mean(0))[:60][min_p]
        elec_clear = 10*np.log10((np.abs(fft(trelec))**2).mean(0))[:60][min_p]

        self.clearance = tr_clear - elec_clear
        print('Clearance at %.1f MHz is %.2f dB' %(freq_clear, self.clearance))

    def import_input_files(self):
        '''
        Import files for input state analysis
        Parameters:
            input_folder: str
                Folder in which data is saved
        Return:
            input_data: dict
                Input files separated by measured phase
            samplehold: dict
                Samples hold files separared by measured phase
            input_meta: list
                Metadata
        '''
        # Import files
        input_files_000 = np.array([os.path.join(self.input_folder, f) for f in sorted(os.listdir(self.input_folder)) if "C1input000" in f])
        input_files_090 = np.array([os.path.join(self.input_folder, f) for f in sorted(os.listdir(self.input_folder)) if "C1input090" in f])
        samplehold_files_000 = np.array([os.path.join(self.input_folder, f) for f in sorted(os.listdir(self.input_folder)) if "C3input000" in f])
        samplehold_files_090 = np.array([os.path.join(self.input_folder, f) for f in sorted(os.listdir(self.input_folder)) if "C3input090" in f])
        elec_files = np.array([os.path.join(self.input_folder, f) for f in sorted(os.listdir(self.input_folder)) if "C1elec" in f])
        
        # Read first file
        input_000 = lecroy.read(input_files_000[0])[2]
        input_090 = lecroy.read(input_files_090[0])[2]
        samplehold_000 = lecroy.read(samplehold_files_000[0])[2]
        samplehold_090 = lecroy.read(samplehold_files_090[0])[2]
        elec = lecroy.read(elec_files[0])[2]

        # Read rest of the files
        if len(input_files_000) > 1:
            for i in range(len(input_files_000)-1):
                input_000 = np.concatenate(input_000, lecroy.read(input_files_000[i+1])[2])
                input_090 = np.concatenate(input_090, lecroy.read(input_files_090[i+1])[2])
                samplehold_000 = np.concatenate(samplehold_000, lecroy.read(samplehold_files_000[i+1])[2])
                samplehold_090 = np.concatenate(samplehold_090, lecroy.read(samplehold_files_090[i+1])[2])

        # Create dictionary
        input_phases = ['000', '090']
        input_data = {'000': input_000, '090': input_090}
        samplehold = {'000': samplehold_000, '090': samplehold_090}
        input_meta = lecroy.read(input_files_000[0])[0]

        self.check_clearance(input_data, elec, plot = False)
        
        return input_data, samplehold, input_meta

    def read_folder(self):
        '''
        Take input state parameters from folder name.
        Parameters:
            input_folder: str
                Folder where input files (or teleported files) are located.
        Return:
            input_alpha: float
                Amplitude of input state
            input_theta: float
                Phase (degrees) of input state
        '''
        #begin_theta = input_folder.find('phase')+len('phase')
        #rest_of_folder = input_folder[begin_theta:]
        #end_theta = begin_theta + rest_of_folder.find('/')
        relay_theta = 90#int(input_folder[begin_theta:end_theta]) # Degrees
        if 'size' in self.input_folder:
            # Get input state values from folder name
            text_size = self.input_folder[self.input_folder.find('size') + len('size'): self.input_folder.find('-')]
            input_angle = 0
            if text_size == '0':
                input_alpha = 0
            else:
                input_alpha = int(text_size[text_size.find('0')+1:])/(10**(len(text_size)-1))
        else:
            input_alpha, input_angle = self.calculate_input_parameters()
        return input_alpha, relay_theta, input_angle

    def calculate_input_parameters(self, block_size = 1000):
        '''
        Compute input state from files.
        Parameters
            input_folder: str
                Folder where input state files are located
            block_size: int
                Size for which data is going to be divided for analysis
        Return
            input_alpha: float
                Amplitude of input state
        '''
        print('CALCULATING INPUT STATE PARAMETERS')
        # Import files
        input_data, samplehold, input_meta = self.import_input_files()
        input_phases = ['000', '090']

        #Time information
        input_sequences = input_meta['subarray_count']
        input_total_sequences = len(input_data['000'])
        input_files_number = int(input_total_sequences/input_sequences)

        input_points_per_seq = int(input_meta['wave_array_count']/input_sequences)
        Ts = input_meta['horiz_interval']
        begin = input_meta['horiz_offset']
        total_points = input_meta['wave_array_count']
        x = np.linspace(begin, begin + Ts*total_points, total_points)
        x_sequence = np.linspace(begin, begin + Ts*input_points_per_seq, input_points_per_seq)

        # Separate input state and vacuum data
        reference = samplehold['000'].mean((0,1))
        vac_indices = {phase: [] for phase in input_phases}
        input_indices = {phase: [] for phase in input_phases}
        for phase in input_phases:
            for sequence in range(input_total_sequences):
                if samplehold[phase][sequence].mean() > reference:
                    vac_indices[phase].append(sequence)
                if samplehold[phase][sequence].mean() < reference:
                    input_indices[phase].append(sequence)
            print(len(vac_indices[phase])/input_total_sequences, 'vacuum files and', len(input_indices[phase])/input_total_sequences, 'input files')
            vac_indices[phase] = np.array(vac_indices[phase])
            input_indices[phase] = np.array(input_indices[phase])

        amount_of_blocks = int(input_total_sequences/block_size)
        block_alpha = []
        block_angle = []
        for block in range(amount_of_blocks):
            mf_vac = {phase: [] for phase in input_phases}
            mf_input = {phase: [] for phase in input_phases}
            for phase in input_phases:
                min_point = block*block_size
                max_point = (block+1)*block_size
                min_vac = vac_indices[phase][vac_indices[phase] > min_point].min()
                min_vac_index = np.where(vac_indices[phase] == min_vac)[0][0]
                max_vac = vac_indices[phase][vac_indices[phase] < max_point].max()
                max_vac_index = np.where(vac_indices[phase] == max_vac)[0][0]
                min_input = input_indices[phase][input_indices[phase] > min_point].min()
                min_input_index = np.where(input_indices[phase] == min_input)[0][0]
                max_input = input_indices[phase][input_indices[phase] < max_point].max()
                max_input_index = np.where(input_indices[phase] == max_input)[0][0]

                reduced_vac = []
                reduced_input = []
                for i, ind in enumerate(vac_indices[phase][min_vac_index:max_vac_index]):
                    reduced_vac.append(input_data[phase][ind])
                for i, ind in enumerate(input_indices[phase][min_input_index:max_input_index]):
                    reduced_input.append(input_data[phase][ind])

                _mf_vac = np.zeros(len(reduced_vac))
                _mf_input = np.zeros(len(reduced_input))
                for k in range(len(reduced_vac)):
                    _mf_vac[k] = np.dot(reduced_vac[k], self.mf)
                for k in range(len(reduced_input)):
                    _mf_input[k] = np.dot(reduced_input[k], self.mf)

                mean_vac = np.mean(_mf_vac)
                var_vac = np.var(_mf_vac)

                mf_vac[phase] = (_mf_vac - mean_vac)/np.sqrt(2*var_vac)
                mf_input[phase] = (_mf_input - mean_vac)/np.sqrt(2*var_vac)

            q_mean = np.mean(mf_input['000'])
            p_mean = np.mean(mf_input['090'])

            angle_rad = np.angle(q_mean+1j*p_mean)
            angle_deg = angle_rad*180/np.pi
            ### Take a better look into this
            input_displacement = np.exp(1j*angle_rad)/np.sqrt(2)*(q_mean+1j*p_mean)
            block_alpha.append(np.abs(input_displacement))
            block_angle.append(angle_deg)

            print('Block', block, 'Input |\\alpha| =', block_alpha[-1])
        input_alpha = np.mean(block_alpha)
        input_angle = np.mean(block_angle)
        print('Input |\\alpha| averaged through all blocks: %.2f (%.2f)' %(input_alpha, np.std(block_alpha)))
        print('Input angle averaged through all blocks: %.2f (%.2f)°' %(input_angle, np.std(block_angle)))

        return input_alpha, input_angle

    def reconstruct_input_state(self):
        self.input_alpha, self.relay_theta, self.input_angle = self.read_folder()

        # Construct density matrix of input state
        input_psi = functions.psi_coherent(self.input_alpha, self.relay_theta + self.input_angle, self.dimension)
        input_rho = functions.rho_psi(input_psi)

        # Plots for the input state
        number = np.linspace(0, self.dimension, self.dimension+1)

        print('Input |alpha| = %.2f'%self.input_alpha)
        print('Relay theta = %.2f°'%self.relay_theta)
        print('Input state theta = %.2f°'%self.input_angle)

        print('PLOTS FOR INPUT STATE')
        if self.input_plot:
            # Input density matrix
            plt.figure(figsize=[6.5,6.5])
            plt.imshow(np.abs(input_rho), norm=matplotlib.colors.Normalize(vmin=-np.max(np.abs(input_rho)), vmax=np.max(np.abs(input_rho))))    
            plt.xlabel("n")
            plt.ylabel("m")
            plt.title("Input density matrix")
            #plt.pause(.05)
            # plt.colorbar()
            plt.show()

            # Input photon number distribution
            figure = plt.figure()
            input_photon_number_distribution = [np.abs(input_rho[n,n]) for n in range(self.dimension+1)]
            plt.bar(number, input_photon_number_distribution)
            plt.xlabel("Photon number")
            plt.ylabel("Probability")
            plt.title("Input state photon number distribution")
            #plt.pause(.05)
            plt.show()

            # Wigner function of the input state
            # Calculate Wigner function
            Q, P, W = functions.wigner_function(input_rho, self.dimension)

            # Plot Wigner function
            W_max = np.max(np.abs(W))

            # Countour plot of Wigner function
            fig = plt.figure(figsize=[9,9])
            ax = plt.axes()
            ax.contourf(Q, P, W, cmap='viridis', norm=matplotlib.colors.Normalize(vmin=-W_max, vmax=W_max))
            ax.set_title('Input state Wigner function')
            ax.set_xlabel('q')
            ax.set_ylabel('p')
            ax.grid()
            fig.add_axes(ax)
            #fig.colorbar(ax)
            plt.show()

        return input_rho

Overwriting inputstate.py


In [4]:
%%writefile coincidencecheck.py

import numpy as np
import matplotlib.pyplot as plt

class selectdata:
    def __init__(self, mf_homodyne, charlie, meta, time_window, time_delay = 95e-9, peak_limit = -.5, tomography_phases = ['000', '030', '060', '090', '120', '150'], verbose = True):
        self.mf_homodyne = mf_homodyne
        self.charlie = charlie
        self.meta = meta
        
        self.peak_limit = peak_limit
        self.time_window = time_window
        self.time_delay = time_delay
        self.tomography_phases = tomography_phases
        self.verbose = verbose
        
        self.sequences = self.meta['subarray_count']
        self.total_sequences = len(charlie[self.tomography_phases[0]])
        self.files_for_each_phase = int(self.total_sequences/self.sequences)
        self.total_points = self.meta['wave_array_count']
        self.points_per_seq = int(self.meta['wave_array_count']/self.sequences)
        self.Ts = self.meta['horiz_interval']
        
    def select_teleported_data(self):
        sequence_with_coincidence, success_rate = self.calculate_success_rate()
        sig = {phase: np.zeros(len(sequence_with_coincidence[phase])) for phase in self.tomography_phases} # Homodyne signal for the data where coincidence was measured
        for phase in self.tomography_phases:
            for s, seq in enumerate(sequence_with_coincidence[phase]):
                sig[phase][s] = self.mf_homodyne[phase][seq]

        return sig, success_rate

    def calculate_success_rate(self):
        coincidence_peaks = {phase: [] for phase in self.tomography_phases}
        sequence_with_coincidence = {phase: [] for phase in self.tomography_phases}
        success_rate = []
        heralding_peak_position = int(self.points_per_seq/2) # The heralding peak is the middle points of the sequence
        # Only search for Charlie peaks in the vicinity defined previously
        first_charlie_point = int(heralding_peak_position + (self.time_delay - self.time_window)/self.Ts)
        last_charlie_point = int(heralding_peak_position + (self.time_delay + self.time_window)/self.Ts) + 1

        for i, phase in enumerate(self.tomography_phases):
        #     peak_limit = np.max(charlie[phase]) - 0.5
            for k in range(len(self.charlie[self.tomography_phases[0]])):
        #        possible_peak = peakdetect(y_axis = np.diff(charlie[phase][k])[first_charlie_point:last_charlie_point], \
        #                                    x_axis = x_sequence[first_charlie_point:last_charlie_point], lookahead = 1)[1]
                d = np.diff(self.charlie[phase][k])[first_charlie_point:last_charlie_point]
                possible_peak = np.argmin(d)
                if d[possible_peak] < self.peak_limit:
                    coincidence_peaks[phase].append(possible_peak)
                    sequence_with_coincidence[phase].append(k)
        #        for l in range(len(possible_peak)):
        #            if possible_peak[l][1] < peak_limit:
        #                coincidence_peaks[phase].append(possible_peak[l])
        #                sequence_with_coincidence[phase].append(k)
            amount = len(coincidence_peaks[phase])
            success_rate.append(amount/len(self.charlie[self.tomography_phases[0]]))
            if self.verbose:
                print('Amount of coincidence clicks for phase %s: %d out of %d' %(phase, amount, self.files_for_each_phase*self.sequences))

        total_success_rate = np.sum(success_rate)/len(self.tomography_phases)
        percentage_of_success = 100*total_success_rate
        print('Success rate = %.2f %%' %percentage_of_success)

        return sequence_with_coincidence, total_success_rate

    def teleported_state_with_optimal_success_rate(self):    

        sequence_with_coincidence, success_rate = self.find_optimal_success_rate()
        sig = {phase: np.zeros(len(sequence_with_coincidence[phase])) for phase in self.tomography_phases} # Homodyne signal for the data where coincidence was measured
        for phase in self.tomography_phases:
            for s, seq in enumerate(sequence_with_coincidence[phase]):
                sig[phase][s] = self.mf_homodyne[phase][seq]

        return sig, success_rate

    def find_optimal_success_rate(self, extra_delay = 5e-9, n_delays = 50):

        # Check which delay yields the maximum amount of coincidences
        heralding_peak_position = int(points_per_seq/2)
        delays = []
        for i in range(n_delays):
            if int(heralding_peak_position + (self.time_delay + extra_delay*(i-n_delays/2) + self.time_window)/Ts) < self.points_per_seq:
                delays.append(extra_delay*(i-n_delays/2))
        delays_str = [str(d) for d in delays]
        dict_sequence_with_coincidence = {d: None for d in delays_str}
        dict_success_rate = {d: None for d in delays_str}
        for d, d_str in enumerate(delays_str):
            if d % 10 == 0:
                print(d)      
            test_delay = self.time_delay + delays[d]
            dict_sequence_with_coincidence[d_str], dict_success_rate[d_str] = self.calculate_success_rate()

        maximum_success_key = max(dict_success_rate, key = dict_success_rate.get)
        #maximum_success_point = np.where(delays_str == maximum_success_key)[0]
        maximum_success_rate = dict_success_rate[maximum_success_key]
        print('Maximum %.2f%% success rate for delay %.0f ns'%(100*maximum_success_rate, float(maximum_success_key)*1e9))

        max_success_rate_list = [dict_success_rate[d] for d in delays_str]
        plt.figure(figsize = [10,8])
        #plt.plot(delays, 100*mean_success)
        plt.plot(delays, 100*np.array(max_success_rate_list))
        plt.xlabel('Extra time delay')
        plt.ylabel('Success rate (%)')
        plt.title('Success rate as a function of time delay (s)')

        return dict_sequence_with_coincidence[maximum_success_key], maximum_success_rate

Overwriting coincidencecheck.py
